## User guide
https://pcse.readthedocs.io/en/stable/user_guide.html#getting-started


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pp
from datetime import datetime

## Running PCSE/WOFOST with custom input data
For running PCSE/WOFOST (and PCSE models in general) with your own data sources you need three different types of inputs:

1. Model parameters that parameterize the different model components. These parameters usually consist of :
   - a set of crop parameters (or multiple sets in case of crop rotations), 
   - a set of soil parameters and 
   - a set of site parameters: ancillary parameters that are specific for a location.

2. Driving variables represented by weather data which can be derived from various sources.
3. Agromanagement actions which specify the farm activities that will take place on the field that is simulated by PCSE.

In [ ]:
from pcse.fileinput import CABOFileReader, YAMLCropDataProvider
from pathlib import Path
from nevergrad.functions.irrigation.common_path import IRRIGATION_DATA_DIR

## 1. Model parameters
### 1.1. Crop parameters

In [ ]:
cropfile = "https://raw.githubusercontent.com/ajwdewit/WOFOST_crop_parameters/master/"
cropdata = YAMLCropDataProvider(repository=cropfile)

In [ ]:
pp(cropdata.get_crops_varieties())

### 1.2. Soil parameters
- soil type 
- soil physical properties

In [ ]:
soilfile = Path(IRRIGATION_DATA_DIR, "soil", "ec3.soil") 
# we will use the water balance for 
# freely draining soils and use the soil file for medium fine sand
soildata = CABOFileReader(soilfile)

In [ ]:
pp(soildata)

### 1.3. Site parameters

In [ ]:
from pcse.util import WOFOST71SiteDataProvider

In [ ]:
# the initial conditions of the water balance such as the initial soil moisture content
sitedata = WOFOST71SiteDataProvider(
    WAV=100, # initial soil moisture content
    CO2=360, # the atmospheric CO2 concentration
)

In [ ]:
expected_sitedata = {
    'IFUNRN': 0,
    'NOTINF': 0,
    'SSI': 0.0, # the initial surface storage
    'SSMAX': 0.0, # the maximum surface storage
    'WAV': 100.0,
    'SMLIM': 0.4
}
assert sitedata == expected_sitedata

### 1.4. Combine them

In [ ]:
from pcse.base import ParameterProvider

In [ ]:
parameters = ParameterProvider(cropdata=cropdata, soildata=soildata, sitedata=sitedata)

## 2. Weather data

In [ ]:
from pcse.db import NASAPowerWeatherDataProvider
from nevergrad.functions.irrigation.irrigation import KNOWN_GEOLOCS

In [ ]:
# address = "Lome"
# geoloc = KNOWN_GEOLOCS.get(address)

# wdp = NASAPowerWeatherDataProvider(
#     latitude=geoloc.get("latitude"), 
#     longitude=geoloc.get("longitude")
# )

In [ ]:
wdp = NASAPowerWeatherDataProvider(
    latitude=51.97, longitude=5.67,
)

In [ ]:
print(wdp)

In [ ]:
crop_start_date = datetime.strptime("2006-03-31", "%Y-%m-%d").date()
crop_end_date = datetime.strptime("2006-10-20", "%Y-%m-%d").date()

In [ ]:
df_weatherdataprovider = pd.DataFrame(wdp.export()).set_index("DAY")

mask = (df_weatherdataprovider.index >= crop_start_date) & (df_weatherdataprovider.index <= crop_end_date)
df_weatherdataprovider.loc[mask, ["RAIN"]].plot()

## 3. Agromanagement

https://github.com/ajwdewit/pcse_notebooks/blob/master/06_advanced_agromanagement_with_PCSE.ipynb

In [ ]:
from pcse.fileinput import YAMLAgroManagementReader
import yaml

In [ ]:
def read_yaml_agro(yaml_agro_file: Path) -> list:
    assert yaml_agro_file.exists()
    with open(yaml_agro_file, "r") as f:
        yaml_agro = f.read()
    return yaml.safe_load(yaml_agro)


In [ ]:
import sys, os
data_dir = os.path.join(os.getcwd(), "data1")
from pcse.fileinput import YAMLAgroManagementReader
agromanagement_file = os.path.join(data_dir, 'agro', 'sugarbeet_calendar.agro')
agromanagement = YAMLAgroManagementReader(agromanagement_file)
print(agromanagement)

In [ ]:
agromanagement_file = os.path.join(data_dir, 'agro', 'sugarbeet_calendar.agro')

In [ ]:
agromanagement = YAMLAgroManagementReader(agromanagement_file)

In [ ]:
agromanagement = yaml.safe_load(yaml_agro_file)

In [ ]:
yaml_agro_file = Path(IRRIGATION_DATA_DIR, "agro", "sugarbeet_calendar.yaml")
agromanagement = read_yaml_agro(yaml_agro_file)


In [ ]:
yaml_agro_irrigation_file = Path(IRRIGATION_DATA_DIR, "agromanagement", "rice_schedule_irrigation.yaml")
agromanagement_irrigation = read_yaml_agro(yaml_agro_irrigation_file)
agromanagement_irrigation

In [ ]:
yaml_agro_irrigation_file = Path(IRRIGATION_DATA_DIR, "agromanagement", "rice_schedule_irrigation_20.yaml")
agromanagement_irrigation_20cm = read_yaml_agro(yaml_agro_irrigation_file)
agromanagement_irrigation_20cm

In [ ]:
event_cfgs = []

first_schedule = agromanagement_irrigation[0]
for campaign_start_date, schedule in first_schedule.items():
    irrigation_events_cfg = schedule['TimedEvents']
    irrigation_events = irrigation_events_cfg[0]["events_table"]
    for event in irrigation_events:
        for event_date, irrigation_water_in_cm in event.items():
            event_cfgs.append((event_date, irrigation_water_in_cm))
            pp([event_date, irrigation_water_in_cm["amount"]])

## 4. Initializing WOFOST model

In [ ]:
from pcse.models import Wofost72_WLP_FD

In [ ]:
wofost_sim = Wofost72_WLP_FD(parameters, wdp, agromanagement)
wofost_irrigation_sim = Wofost72_WLP_FD(parameters, wdp, agromanagement_irrigation)
wofost_irrigation_sim_20cm = Wofost72_WLP_FD(parameters, wdp, agromanagement_irrigation_20cm)

In [ ]:
wofost_sim.run_till_terminate()
wofost_irrigation_sim.run_till_terminate()
wofost_irrigation_sim_20cm.run_till_terminate()

In [ ]:
outputs = wofost_sim.get_output()
df_output = pd.DataFrame(outputs)

outputs_irrigation = wofost_irrigation_sim.get_output()
df_output_irrigation = pd.DataFrame(outputs_irrigation)

outputs_irrigation_20cm = wofost_irrigation_sim_20cm.get_output()
df_output_irrigation_20cm = pd.DataFrame(outputs_irrigation_20cm)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15, 5))
var = "LAI"
df_output.set_index("day")[var].plot(ax=ax)
df_output_irrigation.set_index("day")[var].plot(ax=ax, label=f"{var} irrigation")
#df_output_irrigation_20cm.set_index("day")[var].plot(ax=ax, label=f"{var} irrigation 20cm")

for event_cfg in event_cfgs:
    event_date, irrigation_water_in_cm = event_cfg
    ax.axvline(x=event_date, c="red", label=f"irrigation event: {irrigation_water_in_cm['amount']} cm")

# df_weatherdataprovider.loc[mask, ["RAIN"]].plot(ax=ax)
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15, 5))
var = "SM"
df_output.set_index("day")[var].plot(ax=ax)
df_output_irrigation.set_index("day")[var].plot(ax=ax, label=f"{var} irrigation")
for event_cfg in event_cfgs:
    event_date, irrigation_water_in_cm = event_cfg
    ax.axvline(x=event_date, c="red", label=f"irrigation event: {irrigation_water_in_cm['amount']} cm")

# df_weatherdataprovider.loc[mask, ["RAIN"]].plot(ax=ax)
plt.legend()
plt.show()